In [0]:
import requests 
from datetime import datetime
import time

# Déclaration des variables
API_key="TQLL3YQZLLXNVHGK"
stocks=('AMZN','AAPL','GOOG','MSFT')
#url_api=f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=STOCK&interval=1min&apikey={API_key}&datatype=csv"
#dataSchema = "timestamp timestamp, open float, high float, low float, close float,volume integer"
url_api=f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=STOCK&apikey={API_key}&datatype=csv"
dataSchema = "symbol string, open float, high float, low float, price float,volume integer,latestDay date,previousClose float,change float,changePercent float"

In [0]:
# On arrete tous les streams
for s in spark.streams.active:
  s.stop()
  
# Parcours de la liste des actions
for stock in stocks:
# On supprime les répertoires
    print(f"{stock}")
    dbutils.fs.rm(f"dbfs:/mnt/{stock}/",True)

# Parcours de la liste des actions
for stock in stocks:
# On tente de créer les répertoires
    try:
        dbutils.fs.mkdirs(f"dbfs:/mnt/{stock}")
    except:
        print(f"{stocks[i]} existe déjà")

AMZN
AAPL
GOOG
MSFT


In [0]:
# Parcours de la liste des actions
#while True:
for stock in stocks:
# On appele l'api
    print(f"{stock}")
    response = requests.get(url_api.replace("STOCK",stock.upper()))
# si tout ce passe bien
    if response.ok:
# On enregistre le fichier CSV dans le répertoire concerné
        file_name=f"dbfs:/mnt/{stock}/{stock}-{datetime.now().strftime('%Y%m%d-%H%M')}.csv"
        print(file_name)
        print(response.text.replace("%",""))
        dbutils.fs.put(file_name,response.text.replace("%",""))

        df_stock = (spark.read
                    .option("header", True)
                    .option("inferSchema", True)
                    .csv(file_name))
        display(df_stock)
# On attend 1 minute avant de recommencer
#    time.sleep(60)

AMZN
dbfs:/mnt/AMZN/AMZN-20230707-0715.csv
symbol,open,high,low,price,volume,latestDay,previousClose,change,changePercent
AMZN,128.2500,128.7300,127.3700,128.3600,40697848,2023-07-06,130.3800,-2.0200,-1.5493

Wrote 167 bytes.


symbol,open,high,low,price,volume,latestDay,previousClose,change,changePercent
AMZN,128.25,128.73,127.37,128.36,40697848,2023-07-06,130.38,-2.02,-1.5493


AAPL
dbfs:/mnt/AAPL/AAPL-20230707-0716.csv
symbol,open,high,low,price,volume,latestDay,previousClose,change,changePercent
AAPL,189.8400,192.0200,189.2000,191.8100,45156009,2023-07-06,191.3300,0.4800,0.2509

Wrote 165 bytes.


symbol,open,high,low,price,volume,latestDay,previousClose,change,changePercent
AAPL,189.84,192.02,189.2,191.81,45156009,2023-07-06,191.33,0.48,0.2509


GOOG
dbfs:/mnt/GOOG/GOOG-20230707-0716.csv
symbol,open,high,low,price,volume,latestDay,previousClose,change,changePercent
GOOG,120.6400,121.1499,119.2499,120.9300,17750181,2023-07-06,122.6300,-1.7000,-1.3863

Wrote 167 bytes.


symbol,open,high,low,price,volume,latestDay,previousClose,change,changePercent
GOOG,120.64,121.1499,119.2499,120.93,17750181,2023-07-06,122.63,-1.7,-1.3863


MSFT
dbfs:/mnt/MSFT/MSFT-20230707-0716.csv
symbol,open,high,low,price,volume,latestDay,previousClose,change,changePercent
MSFT,337.3000,342.9900,335.5000,341.2700,28195534,2023-07-06,338.1500,3.1200,0.9227

Wrote 165 bytes.


symbol,open,high,low,price,volume,latestDay,previousClose,change,changePercent
MSFT,337.3,342.99,335.5,341.27,28195534,2023-07-06,338.15,3.12,0.9227


In [0]:
# On crèe les readstreams 
dataPath = f"dbfs:/mnt/GOOG"
DF_GOOG = (spark
    .readStream                            # Returns DataStreamReader
    .option("maxFilesPerTrigger", 1) 
    .option("header",True)      # Force processing of only 1 file per trigger
    .schema(dataSchema)                    # Required for all streaming DataFrames
    .csv(dataPath)                        # The stream's source directory and file type
)

# On crèe les writestreams 
outputPathDir = dataPath + "/output.parquet"                  # A subdirectory for our output
checkpointPath = dataPath + "/checkpoint"                     # A subdirectory for our checkpoint & W-A logs

streamingQuery = (DF_GOOG                                      # Start with our "streaming" DataFrame
  .writeStream                                                # Get the DataStreamWriter
  .queryName("stream_GOOG")                                     # Name the query
  .trigger(processingTime="1 minute")                        # Configure for a 3-second micro-batch
  .format("parquet")                                          # Specify the sink type, a Parquet file
  .option("checkpointLocation", checkpointPath)               # Specify the location of checkpoint files & W-A logs
  .outputMode("append")                                       # Write only new data to the "file"
  .start(outputPathDir)                                       # Start the job, writing to the specified directory
)

In [0]:
# Parcours de la liste des actions
dbutils.fs.ls("dbfs:/mnt/GOOG/output.parquet/")
#dbutils.fs.ls("dbfs:/mnt/AMZN/")
#dbutils.fs.ls("dbfs:/mnt/AAPL/")
#dbutils.fs.ls("dbfs:/mnt/MSFT/")

Out[5]: [FileInfo(path='dbfs:/mnt/GOOG/output.parquet/_spark_metadata/', name='_spark_metadata/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/GOOG/output.parquet/part-00000-2096961c-c2c6-46ba-9113-7e21219615e6-c000.snappy.parquet', name='part-00000-2096961c-c2c6-46ba-9113-7e21219615e6-c000.snappy.parquet', size=2683, modificationTime=1688713957000)]

In [0]:
for s in spark.streams.active:         # Iterate over all streams
  print("{}: {}".format(s.id, s.name)) # Print the stream's id and name

b3ace799-07ad-43a0-86ff-7ce57a6344f8: stream_GOOG


In [0]:
# On charge le parquet 
stock_GOOG = spark.read.parquet(outputPathDir)
# On affiche la structure et le contenu
stock_GOOG.printSchema()
display(stock_GOOG)
# Nombre de lignes
stock_GOOG.count()
stock_GOOG.select('latestDay').distinct().count()
stock_GOOG.write.mode('overwrite').saveAsTable("stock_GOOG")

root
 |-- symbol: string (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- price: float (nullable = true)
 |-- volume: integer (nullable = true)
 |-- latestDay: date (nullable = true)
 |-- previousClose: float (nullable = true)
 |-- change: float (nullable = true)
 |-- changePercent: float (nullable = true)



symbol,open,high,low,price,volume,latestDay,previousClose,change,changePercent
GOOG,120.64,121.1499,119.2499,120.93,17750181,2023-07-06,122.63,-1.7,-1.3863


Databricks visualization. Run in Databricks to view.

In [0]:
%sql

select * from stock_GOOG 

symbol,open,high,low,price,volume,latestDay,previousClose,change,changePercent
GOOG,120.64,121.1499,119.2499,120.93,17750181,2023-07-06,122.63,-1.7,-1.3863
